# 인프런 수강평 스크랩핑

## Selenium 및 웹 드라이버 설치

In [109]:
!pip install Selenium
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

import sys
sys.path.insert(0, '/usr/lib/chromium-browser/chromedriver')

zsh:1: command not found: apt-get
The operation couldn’t be completed. Unable to locate a Java Runtime.
Please visit http://www.java.com for information on installing Java.

cp: /usr/lib/chromium-browser/chromedriver: No such file or directory


## 라이브러리 import

In [158]:
import time
import pandas as pd
import re
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
from selenium.common.exceptions import ElementClickInterceptedException

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('-disable-dev-shm-usage')

In [159]:
def collect_review(reviews, wd) : 
    global review_idx  # 전역 변수로 선언
    for review in reviews:
        author = review.find_element(By.CLASS_NAME,'author').text.split(": ")[1]
        course = review.find_element(By.CLASS_NAME,'relative_course').text.split(": ")[1]
        content = review.find_element(By.CSS_SELECTOR, 'div.post_content > span').text
        course_url = review.find_element(By.CSS_SELECTOR,'div.post_content div.post_metas > a').get_attribute('href')
        datetime = review.find_element(By.CLASS_NAME,'updated_at').get_attribute("datetime")
        
        star_solid = wd.find_element(By.CLASS_NAME,"star_solid").get_attribute("style")  # "width: 100%"
        rating = int(star_solid.split(":")[1][:-2])/20  # 100 /20 = 5점

        

        print(f" 수강평 #{review_idx+1}:", [course, rating, content, author, datetime,course_url])
        review_df.loc[review_idx] = [course, rating, content, author, datetime,course_url]
        review_idx += 1
    
    return review_df


## reivew_scraping() : 리뷰 스크래핑 함수

In [163]:
review_idx = 0

def reivew_scraping(url, wd):
    global review_idx  # 전역 변수로 선언
    review_df = pd.DataFrame(columns=("Course","Rating","Content","Author","DateTime", "CourseURL"))

    reviews = wd.find_elements(By.CLASS_NAME, 'media-content')
    review_df = collect_review(reviews, wd)
    try:
        for i in range(1000):  
            wd.find_element(By.CLASS_NAME, 'pagination-next').click()
            reviews = wd.find_elements(By.CLASS_NAME, 'media-content')
            print("다음페이지")
            review_df.append = collect_review(reviews, wd)
            time.sleep(3)
    
    except ElementClickInterceptedException: 
        time.sleep(3)
        wd.find_element(By.CLASS_NAME, 'pagination-next').click()
        reviews = wd.find_elements(By.CLASS_NAME, 'media-content')
        print("다음페이지")
        review_df.append = collect_review(reviews)
        time.sleep(1)
        
    except: 
        pass
        

    return review_df

In [ ]:
def reivew_scraping(url, wd):
    wd.get(url)
    reviews = wd.find_elements(By.CLASS_NAME, 'media-content')
    review_list = []
    for review in reviews:
        review_list.append(review.text)

    try:
        for i in range(1000):  
            wd.find_element(By.CLASS_NAME, 'pagination-next').click()
            reviews = wd.find_elements(By.CLASS_NAME, 'media-content')
            print("다음페이지")
            for review in reviews:
                review_list.append(review.text)
            time.sleep(3)
    except ElementClickInterceptedException: 
        time.sleep(3)
        wd.find_element(By.CLASS_NAME, 'pagination-next').click()

    return review_list

## scraping() : 스크래핑 함수

In [164]:
def scraping():
    wd = webdriver.Chrome('chromedriver', options = chrome_options)
    wd.implicitly_wait(3)

    url = 'https://www.inflearn.com/community/reviews'
    wd.get(url)

    review_df = reivew_scraping(url, wd)

    wd.close()

    return review_df

In [165]:
review_df = scraping()

 수강평 #1: ['네트워크, 그림으로 이해하자', 5.0, '그림과 함께 수강하게 되어 이해에 도움이 되었습니다.', '권영칠', '2023-04-11T06:17:57.000Z', 'https://www.inflearn.com/course/%EB%84%A4%ED%8A%B8%EC%9B%8C%ED%81%AC-%EA%B7%B8%EB%A6%BC-%EC%9D%B4%ED%95%B4']
 수강평 #2: ['즐거운 자바', 5.0, '기초가 부족해서 다시 처음부터 공부하고 있는데 처음에 알아야할 것들 혹은 좀 더 알두면 좋을 것들을 알려주셔서 좋았어요 !! 나중에 또 공부해야겠어요 ㅎ', '박인성', '2023-04-11T06:13:47.000Z', 'https://www.inflearn.com/course/%EC%A6%90%EA%B1%B0%EC%9A%B4-%EC%9E%90%EB%B0%94']
 수강평 #3: ['ElasticSearch Essential', 5.0, '동작원리 잘 배웠습니다', 'sujeong_lee4', '2023-04-11T06:00:24.000Z', 'https://www.inflearn.com/course/elasticsearch-essential']
 수강평 #4: ['개발 초보를 위한 구체적인 프론트엔드 로드맵 다잇소', 5.0, '강의 잘 들어보았습니다. 섬세하게 처음부터 빠짐없이 취업준비하는 방법을 알려주셔서 도움이 많이 됐습니다. 백엔드부터 프론트까지 로드맵 전자책을 구매했습니다. 로드맵이 구체적이여서 이대로만 따라하면 충분히 중고신입정도 수준으로 회사에서 높게 평가받을 수 있을 것 같습니다. 미리 알았다면 뜬구름 잡지 않고 더 시간을 단축할 수 있었을 것 같아서 지금 알게 되어서 아쉽다는 생각도 듭니다. 부트캠퍼스에 대한 강사님의 평가도 판단이 안서던 제게 강의 선택에 큰 가이드라인이 되어주어 나은 선택을 할 수 있을 것 같습니다. 앞으로도 저런 개발 서비스에 대한 비교 및 평가에 대한 내용이 있으면 초보자로서 판단이 잘 안설 때 판단의

ElementClickInterceptedException: Message: element click intercepted: Element <a class="pagination-next" href="?page=2">...</a> is not clickable at point (722, 593). Other element would receive the click: <div class="el e_el__more ">...</div>
  (Session info: headless chrome=112.0.5615.49)
Stacktrace:
0   chromedriver                        0x0000000102fd9670 chromedriver + 4298352
1   chromedriver                        0x0000000102fd1bbc chromedriver + 4266940
2   chromedriver                        0x0000000102c04758 chromedriver + 280408
3   chromedriver                        0x0000000102c47444 chromedriver + 554052
4   chromedriver                        0x0000000102c44e84 chromedriver + 544388
5   chromedriver                        0x0000000102c4263c chromedriver + 534076
6   chromedriver                        0x0000000102c41530 chromedriver + 529712
7   chromedriver                        0x0000000102c34428 chromedriver + 476200
8   chromedriver                        0x00000

In [166]:
review_df

,Course,Rating,Content,Author,DateTime,CourseURL
0,"네트워크, 그림으로 이해하자",5.0,그림과 함께 수강하게 되어 이해에 도움이 되었습니다.,권영칠,2023-04-11T06:17:57.000Z,https://www.inflearn.com/course/%EB%84%A4%ED%8...
1,즐거운 자바,5.0,기초가 부족해서 다시 처음부터 공부하고 있는데 처음에 알아야할 것들 혹은 좀 더 알...,박인성,2023-04-11T06:13:47.000Z,https://www.inflearn.com/course/%EC%A6%90%EA%B...
2,ElasticSearch Essential,5.0,동작원리 잘 배웠습니다,sujeong_lee4,2023-04-11T06:00:24.000Z,https://www.inflearn.com/course/elasticsearch-...
3,개발 초보를 위한 구체적인 프론트엔드 로드맵 다잇소,5.0,강의 잘 들어보았습니다. 섬세하게 처음부터 빠짐없이 취업준비하는 방법을 알려주셔서 ...,장윤경,2023-04-11T05:58:11.000Z,https://www.inflearn.com/course/%EA%B0%9C%EB%B...
4,스프링 핵심 원리 - 기본편,5.0,"서버 개발을 하고있지만 뭔가 내가 근본적으로 잘 알고 있는게 맞는지,,, 이런 감정...",skywhite15,2023-04-11T05:53:07.000Z,https://www.inflearn.com/course/%EC%8A%A4%ED%9...
5,[하루 10분|Web Project] HTML/JS/CSS로 나만의 심리테스트 사이...,5.0,지루했던 js수업이 mbti와 유형검사 이런 흥미로운 주제로 진행하니까 강의가 지루...,shaaa6256,2023-04-11T05:21:52.000Z,https://www.inflearn.com/course/%EC%8B%AC%EB%A...
6,스프링부트 개념정리(이론),5.0,강의는 너무 훌륭합니다. 다만 아쉬운 것은 관련되어 이어질 수 있는 강의가 없다는 ...,gaabi1204,2023-04-11T05:16:42.000Z,https://www.inflearn.com/course/%EC%8A%A4%ED%9...
7,코딩테스트 [ ALL IN ONE ],5.0,최근 트렌드에는 좀 뒤떨어진느낌,wisehero,2023-04-11T05:13:52.000Z,https://www.inflearn.com/course/%EC%BD%94%EB%9...
8,실습으로 끝장내는 파이썬 웹 크롤링과 웹 페이지 자동화,5.0,물흐르는듯한 설명으로 정말 잘배웠습니다. 특히 이런건 이렇게 하면 됩니다 식의 설명...,김퀴즈,2023-04-11T05:08:31.000Z,https://www.inflearn.com/course/%EC%8B%A4%EC%8...
9,스프링 핵심 원리 - 기본편,5.0,선생님 강의는 항상 최고 에요 b 잘들었습니다,김정훈,2023-04-11T04:59:27.000Z,https://www.inflearn.com/course/%EC%8A%A4%ED%9...


크롤링이 막혀있으므로 우회하여 크롤링하지 않고, 다른 방식으로 수강생들의 반응을 트래킹해보도록 한다.